In [1]:
import pandas as pd
df= pd.read_csv("english_intent.csv")

In [3]:
df["prompt"][0]

'Please categorise the customer intent from the conversation below:\n\n###\n\nCustomer: i waant help to change to the gold account\nThe intention is as follows:'

In [4]:
df["prompt"][1]

'Please categorise the customer intent from the conversation below: \n\n###\n\nCustomer: what do I need to do to shop a  product?\nThe intention is as follows:'

In [5]:
df['prompt'] = df['prompt'].str.split('###\n\nCustomer:').str[-1]
df['prompt'] = 'Customer:' + df['prompt']

print(df)

      Unnamed: 0                                             prompt  \
0              0  Customer: i waant help to change to the gold a...   
1              1  Customer: what do I need to do to shop a  prod...   
2              2  Customer: i bought something cancel purchase 3...   
3              3  Customer: can you show me what hours I can cal...   
4              4  Customer: can I use the Premium account?\nThe ...   
...          ...                                                ...   
1939        1939  Customer: editing account\nThe intention is as...   
1940        1940  Customer: delete Freemium account\nThe intenti...   
1941        1941  Customer: can I write a review for your servic...   
1942        1942  Customer: help seeing order 732201349959 statu...   
1943        1943  Customer: where is ordered 00004587345?\nThe i...   

                         completion  
0                switch account END  
1                   place order END  
2                  cancel order E

In [6]:
df['completion'] = df['completion'].str.replace(' END', '')
df.head()

,Unnamed: 0,prompt,completion
0,0,Customer: i waant help to change to the gold a...,switch account
1,1,Customer: what do I need to do to shop a prod...,place order
2,2,Customer: i bought something cancel purchase 3...,cancel order
3,3,Customer: can you show me what hours I can cal...,contact customer service
4,4,Customer: can I use the Premium account?\nThe ...,switch account


In [8]:
df["prompt"][15]

'Customer: could you help me solve a damn issue with payment?\nThe intention is as follows:'

In [10]:
df['prompt'] = df['prompt'].str.extract(r'(Customer:.*)')


In [11]:
df["prompt"][15]

'Customer: could you help me solve a damn issue with payment?'

In [12]:
df.to_csv("english_intent2.csv")

In [14]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer

data = pd.read_csv("english_intent2.csv")

model_name = 'Helsinki-NLP/opus-mt-tc-big-en-tr'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def translate_to_turkish(text):

    translated = model.generate(**tokenizer(text, return_tensors="pt", padding=True))

    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]


data['prompt_tr'] = data['prompt'].apply(lambda x: translate_to_turkish(x)[0])
data['completion_tr'] = data['completion'].apply(lambda x: translate_to_turkish(x)[0])


output_file_path = 'turkish_intent2.csv'
data.to_csv(output_file_path, index=False)

print(f"Çevrilmiş dosya '{output_file_path}' olarak kaydedildi.")


c:\Users\ASUS\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Çevrilmiş dosya 'turkish_intent2.csv' olarak kaydedildi.


In [16]:
df = pd.read_csv("turkish_intent2.csv")

In [18]:
df.head()

,Unnamed: 0.1,Unnamed: 0,prompt,completion,prompt_tr,completion_tr
0,0,0,Customer: i waant help to change to the gold a...,switch account,Müşteri: Altın hesaba geçmek için yardımdan fe...,hesabı değiştir
1,1,1,Customer: what do I need to do to shop a prod...,place order,Müşteri: Bir ürünü satın almak için ne yapmalı...,yer sırası
2,2,2,Customer: i bought something cancel purchase 3...,cancel order,Müşteri: Bir şey satın aldım satın almayı ipta...,Siparişi iptal et
3,3,3,Customer: can you show me what hours I can cal...,contact customer service,Müşteri: Bana müşteri yardımını hangi saatlerd...,müşteri hizmetleri ile iletişime geçin
4,4,4,Customer: can I use the Premium account?,switch account,Müşteri: Premium hesabı kullanabilir miyim?,hesabı değiştir


In [19]:
df = df[['prompt_tr', 'completion_tr']]

In [20]:
df

,prompt_tr,completion_tr
0,Müşteri: Altın hesaba geçmek için yardımdan fe...,hesabı değiştir
1,Müşteri: Bir ürünü satın almak için ne yapmalı...,yer sırası
2,Müşteri: Bir şey satın aldım satın almayı ipta...,Siparişi iptal et
3,Müşteri: Bana müşteri yardımını hangi saatlerd...,müşteri hizmetleri ile iletişime geçin
4,Müşteri: Premium hesabı kullanabilir miyim?,hesabı değiştir
...,...,...
1939,Müşteri: düzenleme hesabı,Hesabı düzenle
1940,Müşteri: Freemium hesabını sil,hesabı sil
1941,Müşteri: Hizmetleriniz için bir inceleme yazab...,incele
1942,Müşteri: sipariş 732201349959 durumunu görmek ...,parça sırası


In [21]:
df.to_csv("türkçe_intent.csv")